# Build_GNN_2: message passing

In [1]:
import numpy as np
from MPNN_featurize import featurize
from build_GNN_1 import ProteinFeatures, gather_edges, PositionalEncodings

batch = [
    {
        'seq_chain_A': 'MKLVFLVLLVFVQGF',
        'coords_chain_A': {'N_chain_A': np.random.rand(15, 3), 'CA_chain_A': np.random.rand(15, 3), 'C_chain_A': np.random.rand(15, 3), 'O_chain_A': np.random.rand(15, 3)},
        'seq_chain_B': 'MSVKVEEVG',
        'coords_chain_B': {'N_chain_B': np.random.rand(9, 3), 'CA_chain_B': np.random.rand(9, 3), 'C_chain_B': np.random.rand(9, 3), 'O_chain_B': np.random.rand(9, 3)},
        'seq_chain_C': 'ATCGATCGATCGATCG',
        'coords_chain_C': {'N_chain_C': np.random.rand(16, 3), 'CA_chain_C': np.random.rand(16, 3), 'C_chain_C': np.random.rand(16, 3), 'O_chain_C': np.random.rand(16, 3)},
        'masked_list': ['A', 'B'],
        'visible_list': ['C'],
        'num_of_chains': 3,
        'seq': 'MKLVFLVLLVFVQGF'+ 'MSVKVEEVG' + 'ATCGATCGATCGATCG'
    },
      {
        'seq_chain_X': 'ACDEFGHIKLMNPQRSTVWY',
        'coords_chain_X': {'N_chain_X': np.random.rand(20, 3), 'CA_chain_X': np.random.rand(20, 3), 'C_chain_X': np.random.rand(20, 3), 'O_chain_X': np.random.rand(20, 3)},
        'seq_chain_Y': 'ACCDEFGHILKLM',
        'coords_chain_Y': {'N_chain_Y': np.random.rand(13, 3), 'CA_chain_Y': np.random.rand(13, 3), 'C_chain_Y': np.random.rand(13, 3), 'O_chain_Y': np.random.rand(13, 3)},
        'seq_chain_Z': 'LKLMNRPQRST',
        'coords_chain_Z': {'N_chain_Z': np.random.rand(11, 3), 'CA_chain_Z': np.random.rand(11, 3), 'C_chain_Z': np.random.rand(11, 3), 'O_chain_Z': np.random.rand(11, 3)},
        'masked_list': ['X', 'Y'],
        'visible_list': ['Z'],
        'num_of_chains': 3,
        'seq': 'ACDEFGHIKLMNPQRSTVWY'+'ACCDEFGHILKLM'+'LKLMNRPQRST'

    }
]
device='cuda'
X, S, mask, lengths, chain_M, residue_idx, mask_self, chain_encoding_all = featurize(batch, device)
demo = ProteinFeatures(
    edge_features=16, node_features=16, num_positional_embeddings=16,
    num_rbf=16, top_k=30, augment_eps=0., num_chain_embeddings=16
).to(device)
E, E_idx=demo.forward(X,chain_M, residue_idx, chain_encoding_all)

In [2]:
# initialize h_V with zero match h_E dim
import torch
h_V = torch.zeros((E.shape[0], E.shape[1], E.shape[-1]), device=E.device) #[N,L,C]
# MLP of h_E, we omit it here
edge_features=E.shape[-1]
hidden_dim=16
W_e = torch.nn.Linear(edge_features, hidden_dim, bias=True).to(device)
h_E = W_e(E)
print(h_E.shape)

# mask  [B, L]
def gather_nodes(nodes, neighbor_idx):
    # Features [B,L,C] at Neighbor indices [B,L,K] => [B,N,K,C]
    # Flatten and expand indices per batch [B,L,K] => [B,LK] => [B,NK,C]
    neighbors_flat = neighbor_idx.view((neighbor_idx.shape[0], -1)) # [B, LK]
    neighbors_flat = neighbors_flat.unsqueeze(-1).expand(-1, -1, nodes.size(2)) #[B,Lk,1]->[b,lk,c]
    # Gather and re-pack
    # nodes [B,L,L,1]
    neighbor_features = torch.gather(nodes, 1, neighbors_flat) # [b, lk, C]
    neighbor_features = neighbor_features.view(list(neighbor_idx.shape)[:3] + [-1]) # B, L, K, C
    return neighbor_features

# Encoder is unmasked self-attention
mask_attend = gather_nodes(mask.unsqueeze(-1),  E_idx).squeeze(-1) # [B,L,K]
print(mask_attend.shape)
mask_attend = mask.unsqueeze(-1) * mask_attend # [B, L, 1]*[B, L, K] # make sure only true residue has neighours padding mask
print(mask_attend.shape)


torch.Size([2, 44, 30, 16])
torch.Size([2, 44, 30])
torch.Size([2, 44, 30])


# NN ENCODE LAYER
update h_E, h_v using GNN
Here is how they are updated:
message = Mean_Aggreg(3layerMLP(concat(edge feature, neighbours node features))
h_V =  Norm(h_V+mesage))
h_v = Norm( h_V+ MLP(h_V))

Then edge feature h_E is updated based on updated h_V
message = 3layerMLP(concat(edge feature, neighbours node features)
h_E = Norm(h_E + message)


In [3]:
# Original Code
import torch.nn as nn
def gather_nodes(nodes, neighbor_idx):
    # Features [B,N,C] at Neighbor indices [B,N,K] => [B,N,K,C]
    # Flatten and expand indices per batch [B,N,K] => [B,NK] => [B,NK,C]
    neighbors_flat = neighbor_idx.view((neighbor_idx.shape[0], -1))
    neighbors_flat = neighbors_flat.unsqueeze(-1).expand(-1, -1, nodes.size(2))
    # Gather and re-pack
    neighbor_features = torch.gather(nodes, 1, neighbors_flat)
    neighbor_features = neighbor_features.view(list(neighbor_idx.shape)[:3] + [-1])
    return neighbor_features  # [B, N, K, C]

def gather_nodes_t(nodes, neighbor_idx):
    # Features [B,N,C] at Neighbor index [B,K] => Neighbor features[B,K,C]
    idx_flat = neighbor_idx.unsqueeze(-1).expand(-1, -1, nodes.size(2))
    neighbor_features = torch.gather(nodes, 1, idx_flat)
    return neighbor_features

def cat_neighbors_nodes(h_nodes, h_neighbors, E_idx):
    h_nodes = gather_nodes(h_nodes, E_idx)  # [B,L,K,C] 
    h_nn = torch.cat([h_neighbors, h_nodes], -1)
    return h_nn

# not sure the meaning of positionWise but it is a simple 2 layer MLP
class PositionWiseFeedForward(nn.Module):
    def __init__(self, num_hidden, num_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.W_in = nn.Linear(num_hidden, num_ff, bias=True)
        self.W_out = nn.Linear(num_ff, num_hidden, bias=True)
        self.act = torch.nn.GELU()
    def forward(self, h_V):
        h = self.act(self.W_in(h_V))
        h = self.W_out(h)
        return h


class EncLayer(nn.Module):
    def __init__(self, num_hidden, num_in, dropout=0.1, num_heads=None, scale=30):
        super(EncLayer, self).__init__()
        self.num_hidden = num_hidden
        self.num_in = num_in
        self.scale = scale
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)
        self.norm1 = nn.LayerNorm(num_hidden)
        self.norm2 = nn.LayerNorm(num_hidden)
        self.norm3 = nn.LayerNorm(num_hidden)

        self.W1 = nn.Linear(num_hidden + num_in, num_hidden, bias=True)
        self.W2 = nn.Linear(num_hidden, num_hidden, bias=True)
        self.W3 = nn.Linear(num_hidden, num_hidden, bias=True)
        self.W11 = nn.Linear(num_hidden + num_in, num_hidden, bias=True)
        self.W12 = nn.Linear(num_hidden, num_hidden, bias=True)
        self.W13 = nn.Linear(num_hidden, num_hidden, bias=True)
        self.act = torch.nn.GELU()
        self.dense = PositionWiseFeedForward(num_hidden, num_hidden * 4)

    def forward(self, h_V, h_E, E_idx, mask_V=None, mask_attend=None):
        """ Parallel computation of full transformer layer """

        h_EV = cat_neighbors_nodes(h_V, h_E, E_idx) # get neighbers' node feature [B,N,K,C1] and add them to edge feature [B,N,K,c2] ->[B, N, K, C1+c2]
        h_V_expand = h_V.unsqueeze(-2).expand(-1,-1,h_EV.size(-2),-1) # [B,N,C] 2 [B, N, 1, C] 2 [B, N, K, C]
        h_EV = torch.cat([h_V_expand, h_EV], -1) # concat self node feature with neighours' node&edge feature  -> [B, N, K, C1+C2+C3]
        h_message = self.W3(self.act(self.W2(self.act(self.W1(h_EV))))) # 3 layers of MLP to generate message
        if mask_attend is not None:
            h_message = mask_attend.unsqueeze(-1) * h_message
        dh = torch.sum(h_message, -2) / self.scale # sum over K (neighour dimension) then scale-> aggregate message
        h_V = self.norm1(h_V + self.dropout1(dh)) # update h_V with aggragate message

        dh = self.dense(h_V) # add h_V to new MLP to generate new message
        h_V = self.norm2(h_V + self.dropout2(dh)) # add residual and normalize to get final h_V
        if mask_V is not None:
            mask_V = mask_V.unsqueeze(-1)
            h_V = mask_V * h_V

        h_EV = cat_neighbors_nodes(h_V, h_E, E_idx)  # get neighbers' new node feature [B,N,K,C] and add them to edge feature [B,N,K,c]
        h_V_expand = h_V.unsqueeze(-2).expand(-1,-1,h_EV.size(-2),-1)
        h_EV = torch.cat([h_V_expand, h_EV], -1) # [B, N, K, C1+c2+c3]
        h_message = self.W13(self.act(self.W12(self.act(self.W11(h_EV))))) # 3 layer of MLP to generate new edge feature
        h_E = self.norm3(h_E + self.dropout3(h_message)) # add residual  then norm to get final h_E
        return h_V, h_E   # return updated h_V, h_E afte one round of message passing

In [9]:
#h_EV C1+C2+c3=16*3
device='cuda'
num_hidden, num_in=16, 32
test = EncLayer(num_hidden=num_hidden, num_in=num_in).to(device)
h_V, h_E=test.forward (h_V, h_E, E_idx, mask, mask_attend)
print(h_V.shape)
print(h_E.shape)

torch.Size([2, 44, 16])
torch.Size([2, 44, 30, 16])


# Encode layer
updated h_V 

In [14]:
print(S.shape) # N,L

torch.Size([2, 44])


In [ ]:
# Concatenate sequence embeddings for autoregressive decoder
sequence_features=21  # One-hot for AA
W_s = torch.nn.Linear(sequence_features, hidden_dim, bias=True).to(device)
h_S = W_s(S.float())   # N, L, hidden
print(h_S.shape) # N, L, hidden
h_ES = cat_neighbors_nodes(h_S, h_E, E_idx)  #gather neighours' seq feature add to h_E
print(h_ES.shape)
# Build encoder embeddings  initialize X with 0
h_EX_encoder = cat_neighbors_nodes(torch.zeros_like(h_S), h_E, E_idx)
h_EXV_encoder = cat_neighbors_nodes(h_V, h_EX_encoder, E_idx)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x44 and 21x16)

In [5]:
# Decoder layer

class DecLayer(nn.Module):
    def __init__(self, num_hidden, num_in, dropout=0.1, num_heads=None, scale=30):
        super(DecLayer, self).__init__()
        self.num_hidden = num_hidden
        self.num_in = num_in
        self.scale = scale
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.norm1 = nn.LayerNorm(num_hidden)
        self.norm2 = nn.LayerNorm(num_hidden)

        self.W1 = nn.Linear(num_hidden + num_in, num_hidden, bias=True)
        self.W2 = nn.Linear(num_hidden, num_hidden, bias=True)
        self.W3 = nn.Linear(num_hidden, num_hidden, bias=True)
        self.act = torch.nn.GELU()
        self.dense = PositionWiseFeedForward(num_hidden, num_hidden * 4)

    def forward(self, h_V, h_E, mask_V=None, mask_attend=None):
        """ Parallel computation of full transformer layer """

        # Concatenate h_V_i to h_E_ij
        h_V_expand = h_V.unsqueeze(-2).expand(-1,-1,h_E.size(-2),-1)
        h_EV = torch.cat([h_V_expand, h_E], -1)

        h_message = self.W3(self.act(self.W2(self.act(self.W1(h_EV)))))
        if mask_attend is not None:
            h_message = mask_attend.unsqueeze(-1) * h_message
        dh = torch.sum(h_message, -2) / self.scale

        h_V = self.norm1(h_V + self.dropout1(dh))

        # Position-wise feedforward
        dh = self.dense(h_V)
        h_V = self.norm2(h_V + self.dropout2(dh))

        if mask_V is not None:
            mask_V = mask_V.unsqueeze(-1)
            h_V = mask_V * h_V
        return h_V 